In [ ]:
import cv2
import os
import numpy as np
import random
from joblib import Parallel, delayed
import matplotlib.pyplot as plt


In [ ]:

image_dir = 'TB_Chest_Radiography_Database/Normal'
output_dir = 'turberculoseN'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

image_files = os.listdir(image_dir)

for imag_file in image_files:
    imag_P = os.path.join(image_dir,imag_file)
    img = cv2.imread(imag_P)

    if img is None:
        print("Erro")
        continue

    img_Red = cv2.resize(img, (0, 0), fx=0.6, fy=0.6)
    img_g = cv2.GaussianBlur(img_Red, (5, 5), 0)
    gray = cv2.cvtColor(img_g, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    contours, hierarchy = cv2.findContours(opening, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(img, contours, -1, (0, 255, 0), 2)

    output_file = os.path.join(output_dir, f'resized_{imag_file}')
    cv2.imwrite(output_file, opening)


    # img_Red=cv2.resize(img,(0,0), fx=0.6, fy=0.6)
    # #plt.imshow(img_Red)
    # img_g= cv2.GaussianBlur(img_Red,(5,5),0)
    # #plt.imshow(img_g)
    # gray = cv2.cvtColor(img_g, cv2.COLOR_BGR2GRAY)
    # ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # kernel = np.ones((3,3),np.uint8)
    # erosion = cv2.erode(thresh,kernel,iterations = 1)
    # dilation = cv2.dilate(erosion,kernel,iterations = 1)
    # contours, hierarchy = cv2.findContours(dilation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # cv2.drawContours(img, contours, -1, (0,255,0), 2)
    # output_file = os.path.join(output_dir, f'resized_{imag_file}')
    # cv2.imwrite(output_file,dilation)



In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
import os
import random
import shutil

# Definindo diretórios
p1 = 'turberculoseC'
p2 = 'turberculoseN'
train_dir = 'train'
validation_dir = 'validation'

# Criando as pastas train e validation
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(validation_dir):
    os.makedirs(validation_dir)

# Definindo o tamanho do conjunto de treino e validação
train_size = 0.7
validation_size = 0.3

# Mesclando as imagens das pastas p1 e p2 de forma aleatória
images = []
for img_name in os.listdir(p1):
    images.append((os.path.join(p1, img_name), 1))
for img_name in os.listdir(p2):
    images.append((os.path.join(p2, img_name), 0))
random.shuffle(images)

# Copiando as imagens mescladas para as pastas train e validation
for i, (image_path, label) in enumerate(images):
    if i < len(images) * train_size:
        dst_dir = os.path.join(train_dir, str(label))
    else:
        dst_dir = os.path.join(validation_dir, str(label))
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    shutil.copy(image_path, os.path.join(dst_dir, os.path.basename(image_path)))


In [ ]:
#Definindo diretórios
train_dir = 'train'
validation_dir = 'validation'

# Definindo número de classes e tamanho das imagens
num_classes = 2
img_width, img_height = 224, 224

# Definindo o tamanho do batch de imagens a ser utilizado no treinamento
batch_size = 30

# Definindo geradores de imagens para treinamento e validação com aumento de dados
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

# Criando geradores de imagens a partir dos diretórios
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(validation_dir,
                                                       target_size=(img_width, img_height),
                                                       batch_size=batch_size,
                                                       class_mode='categorical')

# Definindo a arquitetura da ResNet50 com pesos pré-treinados no ImageNet
resnet_model = ResNet50(include_top=False, weights='imagenet',
                        input_shape=(img_width, img_height, 3))

# Congelando as camadas convolucionais da ResNet50
for layer in resnet_model.layers:
    layer.trainable = False

# Adicionando camadas densas para classificação
x = resnet_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

# Definindo o modelo final
model = tf.keras.models.Model(inputs=resnet_model.input, outputs=output)

# Compilando o modelo
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Treinando o modelo
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.n // batch_size,
                    epochs=20,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.n // batch_size)

# Avaliando o modelo na validação
scores = model.evaluate(validation_generator)
print(f'Acurácia na validação: {scores[1]*100}%')

# Salvando o modelo treinado
model.save('tb_detection.h5')